# 🧠 CIFAR-10 CNN Experiments

This notebook trains a convolutional neural network (CNN) on the CIFAR-10 dataset
using the utility functions from `utils.py`. The goal is to achieve strong predictive
performance with a clean, modular setup that runs well on a MacBook Air M3.


## 1️⃣ Setup and imports

In this section we import the utilities and configure a few global settings
such as the random seed, batch size and number of epochs.


In [1]:
from src.utils import (
    set_global_seed,
    load_cifar10,
    create_data_augmentation,
    build_cifar10_cnn,
    compile_model,
    train_model,
    evaluate_model,
    predict_classes,
    classification_report_str,
    confusion_matrix_array,
    CLASS_NAMES,
)

import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Ensure reproducibility
set_global_seed(42)

# High-level training configuration
LEARNING_RATE: float = 1e-3
BATCH_SIZE: int = 64
EPOCHS: int = 30  # adjust down if you want faster experiments


## 2️⃣ Load and inspect CIFAR-10 📥

Here we load the CIFAR-10 dataset using the helper from `utils.py` and
briefly inspect shapes and class distribution.


In [2]:
# Load CIFAR-10 data (raw uint8 images, normalization is handled in the model)
data = load_cifar10(normalize=False)

print("Training images:", data.x_train.shape, data.x_train.dtype)
print("Test images:    ", data.x_test.shape, data.x_test.dtype)

# Basic class distribution in the training set
class_counts = np.bincount(data.y_train, minlength=len(CLASS_NAMES))
for idx, (name, count) in enumerate(zip(CLASS_NAMES, class_counts)):
    print(f"Class {idx:2d} ({name:10s}): {count}")


Training images: (50000, 32, 32, 3) float32
Test images:     (10000, 32, 32, 3) float32
Class  0 (airplane  ): 5000
Class  1 (automobile): 5000
Class  2 (bird      ): 5000
Class  3 (cat       ): 5000
Class  4 (deer      ): 5000
Class  5 (dog       ): 5000
Class  6 (frog      ): 5000
Class  7 (horse     ): 5000
Class  8 (ship      ): 5000
Class  9 (truck     ): 5000


### 📊 Class distribution plot

The bar chart below shows how many samples we have per class in the
training split. CIFAR-10 is perfectly balanced, which is helpful for
both accuracy and recall.


In [4]:
fig_class_dist = px.bar(
    x=CLASS_NAMES,
    y=class_counts,
    title="CIFAR-10 training set class distribution",
    labels={"x": "Class", "y": "Count"},
)
fig_class_dist.update_layout(xaxis_tickangle=0)
fig_class_dist.show()


## 3️⃣ Build and compile the CNN 🧱

We now create a reasonably strong CNN architecture with:

- Data augmentation (random flips, rotations, zoom)
- Convolutional blocks with Batch Normalization and ReLU
- Max pooling and Dropout for downsampling and regularization
- A dense classification head with Dropout before the softmax layer

Compilation uses the Adam optimizer with a sensible learning rate for CIFAR-10.


In [5]:
# Create data augmentation pipeline
data_augmentation = create_data_augmentation()

# Build model
model = build_cifar10_cnn(
    input_shape=data.x_train.shape[1:],
    num_classes=len(CLASS_NAMES),
    data_augmentation=data_augmentation,
)

# Compile model
compile_model(model, learning_rate=LEARNING_RATE)

model.summary()


Model: "cifar10_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 128)      │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 323,946 (1.24 MB)

 Trainable params: 323,050 (1.23 MB)

 Non-trainable params: 896 (3.50 KB)

## 4️⃣ Train the model 🚂

We train the model on the full CIFAR-10 training set with a small
validation split to monitor generalization. Default callbacks from
`train_model` use learning rate scheduling and early stopping
to reach strong performance without excessive overfitting.


In [7]:
history = train_model(
    model,
    data.x_train,
    data.y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
)


Epoch 1/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 87s 123ms/step - accuracy: 0.4918 - loss: 1.3944 - val_accuracy: 0.3986 - val_loss: 2.1784 - learning_rate: 0.0010
Epoch 2/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 127s 180ms/step - accuracy: 0.5549 - loss: 1.2374 - val_accuracy: 0.4606 - val_loss: 1.6827 - learning_rate: 0.0010
Epoch 3/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 103s 146ms/step - accuracy: 0.5950 - loss: 1.1360 - val_accuracy: 0.5238 - val_loss: 1.4734 - learning_rate: 0.0010
Epoch 4/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 108s 154ms/step - accuracy: 0.6244 - loss: 1.0705 - val_accuracy: 0.5104 - val_loss: 1.6147 - learning_rate: 0.0010
Epoch 5/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 103s 147ms/step - accuracy: 0.6440 - loss: 1.0121 - val_accuracy: 0.6180 - val_loss: 1.2083 - learning_rate: 0.0010
Epoch 6/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 109s 154ms/step - accuracy: 0.6636 - loss: 0.9627 - val_accuracy: 0.6170 - val_loss: 1.2408 - learning_rate: 0.0010
Epoch 7/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 107s 152ms/step - accurac

In [8]:
import os
from tensorflow import keras
import json

os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

# 1) Modell speichern
model.save("models/cifar10_main.keras")

# 2) Training-History speichern (für Plots später)
with open("results/history_cifar10_main.json", "w") as f:
    json.dump(history.history, f)

print("Saved model to models/cifar10_main.keras")
print("Saved history to results/history_cifar10_main.json")

Saved model to models/cifar10_main.keras
Saved history to results/history_cifar10_main.json


## 5️⃣ Visualize training curves 📉

The following plots show training and validation accuracy and loss over
epochs. They help to understand convergence speed and potential
overfitting or underfitting.


In [10]:
history_dict = history.history
epochs_range = list(range(1, len(history_dict["loss"]) + 1))

# Accuracy curves
fig_acc = go.Figure()
fig_acc.add_trace(
    go.Scatter(
        x=epochs_range,
        y=history_dict["accuracy"],
        mode="lines+markers",
        name="Train accuracy",
    ),
)
fig_acc.add_trace(
    go.Scatter(
        x=epochs_range,
        y=history_dict["val_accuracy"],
        mode="lines+markers",
        name="Val accuracy",
    ),
)
fig_acc.update_layout(
    title="Training and validation accuracy",
    xaxis_title="Epoch",
    yaxis_title="Accuracy",
)
fig_acc.show()

# Loss curves
fig_loss = go.Figure()
fig_loss.add_trace(
    go.Scatter(
        x=epochs_range,
        y=history_dict["loss"],
        mode="lines+markers",
        name="Train loss",
    ),
)
fig_loss.add_trace(
    go.Scatter(
        x=epochs_range,
        y=history_dict["val_loss"],
        mode="lines+markers",
        name="Val loss",
    ),
)
fig_loss.update_layout(
    title="Training and validation loss",
    xaxis_title="Epoch",
    yaxis_title="Loss",
)
fig_loss.show()


## 6️⃣ Evaluate on the test set ✅

Finally we evaluate the trained model on the CIFAR-10 test set and
inspect both aggregate metrics and a per-class classification report.


In [11]:
# Evaluate on test data
metrics = evaluate_model(model, data.x_test, data.y_test)
print(f"Test loss:     {metrics['loss']:.4f}")
print(f"Test accuracy: {metrics['accuracy']:.4f}")

# Predict test labels
y_test_pred = predict_classes(model, data.x_test)

# Classification report (precision / recall / F1 per class)
report_str = classification_report_str(data.y_test, y_test_pred, target_names=CLASS_NAMES)
print(report_str)


Test loss:     0.5797
Test accuracy: 0.8093
              precision    recall  f1-score   support

    airplane     0.8536    0.7870    0.8189      1000
  automobile     0.8901    0.9310    0.9101      1000
        bird     0.8093    0.7300    0.7676      1000
         cat     0.8171    0.5270    0.6407      1000
        deer     0.8231    0.7400    0.7794      1000
         dog     0.8368    0.6820    0.7515      1000
        frog     0.6948    0.9400    0.7990      1000
       horse     0.7811    0.9030    0.8377      1000
        ship     0.8658    0.9100    0.8874      1000
       truck     0.7787    0.9430    0.8530      1000

    accuracy                         0.8093     10000
   macro avg     0.8150    0.8093    0.8045     10000
weighted avg     0.8150    0.8093    0.8045     10000



## 7️⃣ Confusion matrix 🔍

The normalized confusion matrix below shows how often each true class
is predicted as each target class. Bright diagonal values indicate
good per-class performance, while off-diagonal values highlight
systematic confusions between classes.


In [13]:
# Normalized confusion matrix
cm_norm = confusion_matrix_array(data.y_test, y_test_pred, normalize=True)

fig_cm = px.imshow(
    cm_norm,
    x=CLASS_NAMES,
    y=CLASS_NAMES,
    color_continuous_scale="Viridis",
    labels={"x": "Predicted class", "y": "True class", "color": "Proportion"},
    title="Normalized confusion matrix (CIFAR-10)",
)
fig_cm.update_xaxes(side="top")
fig_cm.show()
